In [1]:
import json
import os
import grid2op
import numpy as np
from collections import defaultdict
from grid2op.PlotGrid import PlotMatplot
from lightsim2grid import LightSimBackend
from grid2op.Agent import DoNothingAgent
from grid2op.Runner import Runner
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm.auto import tqdm  # for easy progress bar
import pandas as pd
from grid2op.Episode import EpisodeData
from ray.rllib.algorithms import ppo
from rl4pnc.evaluation.evaluation_agents import RllibAgent
from rl4pnc.grid2op_env.custom_environment import CustomizedGrid2OpEnvironment
from rl4pnc.grid2op_env.custom_env2 import RlGrid2OpEnv
from rl4pnc.experiments.yaml import load_config
from rl4pnc.evaluation.utils import instantiate_reward_class
from rl4pnc.grid2op_env.utils import load_actions

In [2]:
main_folder = "/Users/ericavandersar/surfdrive/Documents/Research/Result/Case14_Sandbox_ActSpaces/"
agent_name = "CustomPPO_RlGrid2OpEnv_6fdf8_00000_0_2024-05-06_05-12-14"
data_folder = os.path.join(main_folder, agent_name, "evaluation_episodes")
df = pd.read_csv(os.path.join(data_folder, "line_action_topo_data.csv"))
df

,Unnamed: 0,chron_id,ts_danger,line_danger,rho,action_sub,action_topo,subs_changed,sub_topo_depth,el_changed,el_topo_depth
0,0,20,513,17,0.955241,4,"[2, 1, 2, 2, 1]",[4],1,[19 21 22],3
1,1,20,514,17,0.950289,8,"[1, 1, 1, 2, 2]",[4 8],2,[19 21 22 39 40],5
2,2,20,2533,17,0.953320,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
3,3,20,6741,9,0.951646,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
4,4,20,6742,9,0.959878,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
5,5,20,6743,9,0.956162,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
6,6,20,6744,9,0.965621,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
7,7,20,6745,9,0.973284,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
8,8,20,6746,9,0.963417,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9
9,9,20,6747,9,0.966929,3,"[2, 2, 1, 1, 2, 2]",[3 4 8],3,[13 14 17 18 19 21 22 39 40],9


In [120]:
df.value_counts("line_danger").reset_index(name='frequency')

,line_danger,frequency
0,9,685
1,16,436
2,17,244
3,4,26
4,7,8


In [124]:
dfg = df.groupby(by=["action_sub", "action_topo"], as_index=False).size()
dfg

,action_sub,action_topo,size
0,3,"[2, 2, 1, 1, 2, 2]",1141
1,3,"[2, 2, 1, 2, 1, 2]",53
2,4,"[2, 1, 2, 2, 1]",103
3,8,"[1, 1, 1, 2, 2]",102


In [49]:
df_sub_act = df.value_counts(["action_sub", "action_topo"]).reset_index(name='frequency')
df_sub_act

,action_sub,action_topo,frequency
0,8,"[1, 1, 1, 2, 2]",1512
1,1,"[2, 2, 1, 2, 1, 2]",191
2,4,"[2, 1, 1, 2, 2]",123


In [121]:
def collect_episode_data(env, store_trajectories_folder, li_episode):
    print(" Start collecting episode data ... ")
    chron = []
    surv = []
    
    for ep in tqdm(li_episode, total=len(li_episode)):
        full_path, episode_studied = ep
        this_episode = EpisodeData.from_disk(store_trajectories_folder, episode_studied)
        surv.append(this_episode.meta['nb_timestep_played'])
        chron.append(os.path.basename(os.path.normpath(this_episode.meta['chronics_path'])))
    all_data = {'chron': chron, 'survived': surv} 
    df_surv = pd.DataFrame(all_data)
    print(df_surv.head())
    df_surv.to_csv(os.path.join(store_trajectories_folder, "survival.csv"), index=False)
    return df_surv

In [122]:
env = grid2op.make("l2rpn_case14_sandbox", backend=LightSimBackend())
li_episode = EpisodeData.list_episode(data_folder)
df_sur = collect_episode_data(env, data_folder, li_episode)
df_sur

 Start collecting episode data ... 


  0%|          | 0/100 [00:00<?, ?it/s]

  chron  survived
0  0020      8064
1  0023      8064
2  0026      8064
3  0030      8064
4  0047      8064


,chron,survived
0,0020,8064
1,0023,8064
2,0026,8064
3,0030,8064
4,0047,8064
5,0065,8064
6,0066,8064
7,0075,8064
8,0076,8064
9,0103,8064


In [6]:
env = grid2op.make("l2rpn_case14_sandbox", backend=LightSimBackend())
li_episode = EpisodeData.list_episode(data_folder)
full_path, episode_studied = li_episode[0]
this_episode = EpisodeData.from_disk(data_folder, episode_studied)
this_episode.META

'episode_meta.json'

In [7]:
this_episode.meta

{'agent_seed': None,
 'backend_type': 'LightSimBackend_l2rpn_case14_sandbox',
 'chronics_max_timestep': '8064',
 'chronics_path': '/Users/ericavandersar/data_grid2op/l2rpn_case14_sandbox/chronics/0020',
 'cumulative_reward': 507867.53125,
 'env_seed': None,
 'env_type': 'Environment_l2rpn_case14_sandbox',
 'grid_path': '/Users/ericavandersar/data_grid2op/l2rpn_case14_sandbox/grid.json',
 'nb_timestep_played': 8064}

In [130]:
def actions_per_agent(agents, main_folder, reset_topo=False):
    data_dict = {}
    for agent_name in agents:
        # Get agent code
        agent_code = agent_name.split("_")[2]

        # Get action space
        agent_path = os.path.join(main_folder, agent_name)
        with open(os.path.join(agent_path,'params.json')) as json_file:
            agent_pars = json.load(json_file)
        data_dict[agent_code] = {"action_space": agent_pars["env_config"]["action_space"]}
        
        # Timesteps survived
        if reset_topo:
            data_folder = os.path.join(agent_path, "evaluation_episodes_testreset")
        else:
            data_folder = os.path.join(agent_path, "evaluation_episodes")
        df_surv = pd.read_csv(os.path.join(data_folder, "survival.csv"))
        data_dict[agent_code]["mean_surv_ts"] = df_surv.survived.mean()

        
        # Size action space
        lib_dir = "/Users/ericavandersar/Documents/Python_Projects/Research/rl4pnc"
        path = os.path.join(
                lib_dir,
                f"data/action_spaces/{env.name}/{act_space}.json",
            )
        actions = load_actions(path, env)
        data_dict[agent_code]["n_actions"] = len(actions)
        
        # action frequency
        df = pd.read_csv(os.path.join(data_folder, "line_action_topo_data.csv"))
        df_sub_act = df.value_counts(["action_sub", "action_topo"]).reset_index(name='frequency')
        print(df_sub_act)
        for idx, row in df_sub_act.iterrows():
            data_dict[agent_code][f'sub{row["action_sub"]}_{row["action_topo"]}'] = row["frequency"]
        df_line_danger = df.value_counts("line_danger").reset_index(name='frequency')
        for idx, row in df_line_danger.iterrows():
            data_dict[agent_code][f'line_{row["line_danger"]}'] = row["frequency"]
    new_df = pd.DataFrame.from_dict(data_dict, orient='index')
    new_df = new_df[
    ['action_space', 'mean_surv_ts', 'n_actions'] 
    + sorted([col for col in new_df if col.startswith('sub')]) 
    + [col for col in new_df if col.startswith('line')]
    ]
    new_df.to_csv(os.path.join(main_folder, "agents_overview.csv"), index=False)
    # order columns:
    return new_df

In [39]:
lib_dir = "/Users/ericavandersar/Documents/Python_Projects/Research/rl4pnc"
path = os.path.join(
        lib_dir,
        f"data/action_spaces/{env.name}/{act_space}.json",
    )
possible_substation_actions = load_actions(path, env)
print('number actions: ', len(possible_substation_actions))

number actions:  140


In [50]:
df_sub_act

,action_sub,action_topo,frequency
0,8,"[1, 1, 1, 2, 2]",1512
1,1,"[2, 2, 1, 2, 1, 2]",191
2,4,"[2, 1, 1, 2, 2]",123


In [54]:
for idx, row in df_sub_act.iterrows():
    sub_act = f'sub{row["action_sub"]}_{row["action_topo"]}'
    freq = row["frequency"]
    print(f"substation actions {sub_act} frequency {freq}")

substation actions sub8_[1, 1, 1, 2, 2] frequency 1512
substation actions sub1_[2, 2, 1, 2, 1, 2] frequency 191
substation actions sub4_[2, 1, 1, 2, 2] frequency 123


In [55]:
df_sur.survived.mean()

8064.0

In [131]:
agents = [
    "CustomPPO_RlGrid2OpEnv_736a5_00000_0_2024-05-06_03-10-39",
    "CustomPPO_RlGrid2OpEnv_7d675_00000_0_2024-05-06_10-20-25",
    "CustomPPO_RlGrid2OpEnv_6fdf8_00000_0_2024-05-06_05-12-14",
    ]
df_res = actions_per_agent(agents, main_folder)
df_res

   action_sub         action_topo  frequency
0           8     [1, 1, 1, 2, 2]       1512
1           1  [2, 2, 1, 2, 1, 2]        191
2           4     [2, 1, 1, 2, 2]        123
   action_sub         action_topo  frequency
0           8     [1, 1, 1, 2, 2]        510
1           4     [2, 1, 1, 2, 2]        297
2           3  [2, 2, 1, 2, 1, 2]        143
3           1  [2, 2, 1, 1, 1, 2]         32
4          11           [1, 1, 1]          1
   action_sub         action_topo  frequency
0           3  [2, 2, 1, 1, 2, 2]       1141
1           4     [2, 1, 2, 2, 1]        103
2           8     [1, 1, 1, 2, 2]        102
3           3  [2, 2, 1, 2, 1, 2]         53


,action_space,mean_survived_ts,n_actions,"sub11_[1, 1, 1]","sub1_[2, 2, 1, 1, 1, 2]","sub1_[2, 2, 1, 2, 1, 2]","sub3_[2, 2, 1, 1, 2, 2]","sub3_[2, 2, 1, 2, 1, 2]","sub4_[2, 1, 1, 2, 2]","sub4_[2, 1, 2, 2, 1]","sub8_[1, 1, 1, 2, 2]",line_9,line_17,line_16,line_7,line_4
736a5,medha_DN_rev,8064.00,140,NaN,NaN,191.0,NaN,NaN,123.0,NaN,1512,1372,225,153,43,33
7d675,medha_DN_rev,7854.53,140,1.0,32.0,NaN,NaN,143.0,297.0,NaN,510,357,396,128,3,99
6fdf8,medha_dn_optshunt,7717.06,140,NaN,NaN,NaN,1141.0,53.0,NaN,103.0,102,685,244,436,8,26


In [3]:
env = grid2op.make("l2rpn_case14_sandbox", backend=LightSimBackend())
lib_dir = "/Users/ericavandersar/Documents/Python_Projects/Research/rl4pnc"
act_space = "medha_dn_allshunt_maxrho1.0-30-100_gr"
path = os.path.join(
    lib_dir,
    f"data/action_spaces/{env.name}/{act_space}.json",
)
actions = load_actions(path, env)

In [21]:
action_dict = defaultdict(list)
for action in actions:
    act = action.set_bus
    if any(act!=0):
        print(f" Substation {env._topo_vect_to_sub[act != 0][0]} - topo config: {list(act[act != 0].astype(int))}" )
        action_dict[f"substation {env._topo_vect_to_sub[act != 0][0]}"].append(list(act[act != 0].astype(int)))
    else: 
        print(15) 

 Substation 0 - topo config: [1, 1, 1]
 Substation 1 - topo config: [2, 2, 1, 2, 1, 2]
 Substation 3 - topo config: [2, 1, 2, 2, 2, 1]
 Substation 3 - topo config: [2, 1, 1, 2, 2, 1]
 Substation 3 - topo config: [2, 1, 1, 1, 1, 2]
 Substation 4 - topo config: [2, 1, 2, 2, 1]
 Substation 4 - topo config: [2, 1, 2, 1, 2]
 Substation 4 - topo config: [2, 1, 1, 2, 2]
 Substation 5 - topo config: [1, 1, 1, 1, 1, 1, 1]
 Substation 5 - topo config: [2, 1, 2, 2, 1, 2, 2]
 Substation 5 - topo config: [2, 1, 1, 1, 1, 2, 1]
 Substation 8 - topo config: [1, 1, 1, 1, 1]
 Substation 9 - topo config: [1, 1, 1]
 Substation 8 - topo config: [1, 1, 1, 2, 2]
 Substation 8 - topo config: [2, 2, 2, 1, 1]
 Substation 1 - topo config: [1, 1, 1, 1, 1, 1]
 Substation 3 - topo config: [1, 1, 1, 1, 1, 1]
 Substation 4 - topo config: [1, 1, 1, 1, 1]


In [25]:
action_dict

defaultdict(list,
            {'substation 0': [[1, 1, 1]],
             'substation 1': [[2, 2, 1, 2, 1, 2], [1, 1, 1, 1, 1, 1]],
             'substation 3': [[2, 1, 2, 2, 2, 1],
              [2, 1, 1, 2, 2, 1],
              [2, 1, 1, 1, 1, 2],
              [1, 1, 1, 1, 1, 1]],
             'substation 4': [[2, 1, 2, 2, 1],
              [2, 1, 2, 1, 2],
              [2, 1, 1, 2, 2],
              [1, 1, 1, 1, 1]],
             'substation 5': [[1, 1, 1, 1, 1, 1, 1],
              [2, 1, 2, 2, 1, 2, 2],
              [2, 1, 1, 1, 1, 2, 1]],
             'substation 8': [[1, 1, 1, 1, 1],
              [1, 1, 1, 2, 2],
              [2, 2, 2, 1, 1]],
             'substation 9': [[1, 1, 1]]})

In [42]:
action = actions[1].set_bus
action

array([0, 0, 0, 2, 2, 1, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
obs = env.reset()
obs.topo_vect

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [43]:
action[action!=0]

array([2, 2, 1, 2, 1, 2], dtype=int32)

In [44]:
obs.topo_vect[action!=0]

array([1, 1, 1, 1, 1, 1], dtype=int32)

In [46]:
env.current_obs.topo_vect

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [49]:
np.all(obs.topo_vect[action!=0] == action[action!=0])

False